In [1]:
import pandas as pd

path = "/home/simon/Documents/data_science/Thesis/benchmark_truth/aktiva_passiva_guv_table_pages.csv"

df = pd.read_csv(path, sep=",")
df_aktiva = df[df['type'] == 'Aktiva']
df_aktiva

,filepath,page,type
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
6,../Geschaeftsberichte/Berliner Bäder Betriebe/...,16,Aktiva
9,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
12,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
...,...,...,...
251,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,40,Aktiva
254,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,Aktiva
257,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,40,Aktiva
260,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,42,Aktiva


In [2]:
from pypdf import PdfReader, PdfWriter

output_pdf = PdfWriter()

for idx, row in df_aktiva.iterrows():
    pdf_path = row['filepath']
    page_num = row['page'] - 1  # PyPDF2 uses 0-based indexing
    try:
        reader = PdfReader('../'+pdf_path)
        output_pdf.add_page(reader.pages[page_num])
    except Exception as e:
        print(f"Error processing {pdf_path} page {row['page']}: {e}")

with open("aktiva_pages_combined.pdf", "wb") as f_out:
    output_pdf.write(f_out)

In [ ]:
from marker.converters.table import TableConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = TableConverter(
    artifact_dict=create_model_dict(),
)

Loaded layout model s3://layout/2025_02_18 on device cpu with dtype torch.float32
Loaded texify model s3://texify/2025_02_18 on device cpu with dtype torch.float32
Loaded recognition model s3://text_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded table recognition model s3://table_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded detection model s3://text_detection/2025_02_28 on device cpu with dtype torch.float32
Loaded detection model s3://inline_math_detection/2025_02_24 on device cpu with dtype torch.float32


In [7]:
import os

# print(os.getcwd())

file_path = './separate_files/aktiva_table_4_columns_span_False_thin_True_year_as_date_unit_in_first_cell_False_EUR_enumeration_True_0'
rendered = converter(file_path+'.pdf')
text, _, images = text_from_rendered(rendered)

Running OCR Error Detection: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


In [10]:
print(rendered.markdown)

| Aktiva                                                                                                                                                                                        | 31.12.2013   | 31.12.2013    | 31.12.2012    |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------|---------------|---------------|
|                                                                                                                                                                                               | EUR          | EUR           | EUR           |
| A.<br>Anlagevermögen                                                                                                                                                                          |              |               |               |
| I.<br>Immaterielle<br>Vermögensgeg

In [ ]:
import pandas as pd

file_path = './separate_files/aktiva_table_4_columns_span_False_thin_True_year_as_date_unit_in_first_cell_False_EUR_enumeration_True_0'
df = pd.read_csv(file_path+'.csv')

In [12]:
from pprint import pprint

print(df.dropna(subset=[df.columns[-1]]).to_markdown(index=False, floatfmt=".2f"))

| E1                                                      | E2                                                                           | E3                                                                                                                                      |   31.12.2013 |   31.12.2012 |
|:--------------------------------------------------------|:-----------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------|-------------:|-------------:|
| Anlagevermögen                                          | Immaterielle Vermögensgegenstände                                            | geleistete Anzahlungen                                                                                                                  |    913637.50 |   5560454.30 |
| Anlagevermögen                                          | Immaterielle Vermög

In [14]:
file_path.split('_')[-4]

'EUR'